In [61]:
!pip install keras

In [62]:
import datalab.bigquery as bq
import seaborn as sns
import pandas as pd
import numpy as np
import shutil
import os
import h5py
from sklearn import preprocessing
import keras
from pandas_gbq.gbq import GenericGBQException
import logging
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
import tensorflow as tf
print tf.__version__

1.5.0


In [63]:
%sql --module letslookatthedata
SELECT * FROM [go-de-internal:democases_go_reply_internal.churn_not_time_dependent] LIMIT 10000

In [64]:
churn = bq.Query(letslookatthedata).to_dataframe()
churn.describe()

,timezone_cd,tenure_date_created,tenure_visit,branch,abc_cd,abc_detailed_cd,avg_no_main_cat_1y,avg_no_main_cat_q1,avg_no_main_cat_q2,avg_no_main_cat_th,...,ret_visits_q1,ret_visits_q2,ret_visits_th,ret_visits_lh,ret_sales_q1,ret_sales_q2,ret_sales_th,ret_sales_lh,ret_sales_1y,prediction_week
count,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.239400,2369.479300,353.39320,1.665700,1.913700,6.913300,17.258491,16.740775,17.890048,17.172363,...,2.647900,2.230000,4.877900,4.81720,214.295305,158.347319,372.642624,305.503523,678.146147,26.438400
std,2.289369,1568.307173,14.21464,0.894552,0.280821,1.389452,7.632910,8.013448,8.585262,7.851447,...,2.147647,1.450417,3.018858,4.15359,1336.707169,865.855804,1703.943625,1345.410025,2532.920120,15.548556
min,-1.000000,274.000000,274.00000,1.000000,1.000000,1.000000,1.454545,1.363636,1.000000,1.315789,...,1.000000,1.000000,2.000000,0.00000,0.010000,0.010000,0.970000,0.000000,0.970000,1.000000
25%,-1.000000,1226.750000,351.00000,1.000000,2.000000,6.000000,12.095727,11.037037,11.800000,11.623512,...,1.000000,1.000000,3.000000,2.00000,9.150000,8.290000,38.890000,20.740000,105.427500,13.000000
50%,1.000000,2394.000000,358.00000,1.000000,2.000000,7.000000,16.359127,15.636364,16.816667,16.300000,...,2.000000,2.000000,4.000000,4.00000,40.355000,36.710000,120.725000,78.680000,241.860000,26.000000
75%,3.000000,3065.000000,361.00000,3.000000,2.000000,8.000000,21.627907,21.287815,22.888889,21.440000,...,3.000000,3.000000,6.000000,6.00000,139.047500,128.640000,279.132500,226.131000,503.050000,39.000000
max,7.000000,13850.000000,364.00000,3.000000,2.000000,8.000000,53.648649,65.625000,65.625000,61.500000,...,55.000000,10.000000,58.000000,58.00000,47918.180000,29622.640000,47922.040000,30750.830000,47933.620000,53.000000


In [ ]:
def query_gbq(query, project_id, **bq_query_config_args):
    try:
        data = pd.read_gbq(
            query, project_id,
            configuration=get_bq_query_config(project_id, **bq_query_config_args)
        ) # 
    except GenericGBQException:
        logging.warning(
            "Generic GBQ exception detected, probably because the intermediate data table already exists,"
            "attempting to overwrite...")
        query_config = get_bq_query_config(project_id, **bq_query_config_args)
        query_config['query']["writeDisposition"] = "WRITE_TRUNCATE" # tells BQ to overwrite
        data = pd.read_gbq(query, project_id, configuration=query_config)
        
    return data

def get_bq_query_config(
    projectId, allowLargeResults=True, datasetId='democases_go_reply_internal', tableId='churn_not_time_dependent'
):
    return {"query": {
        "allowLargeResults": allowLargeResults,
        'destinationTable': {
            'projectId': projectId,
            'datasetId': datasetId,
            'tableId': tableId
        }
    }}

churn = query_gbq(
    "SELECT * FROM democases_go_reply_internal.churn_not_time_dependent LIMIT 100000000", 
    "go-de-internal")

In [65]:
churn.head()

,timezone_cd,tenure_date_created,tenure_visit,branch,abc_cd,abc_detailed_cd,avg_no_main_cat_1y,avg_no_main_cat_q1,avg_no_main_cat_q2,avg_no_main_cat_th,...,ret_visits_q1,ret_visits_q2,ret_visits_th,ret_visits_lh,ret_sales_q1,ret_sales_q2,ret_sales_th,ret_sales_lh,ret_sales_1y,prediction_week
0,5,2244,361,1,1,4,11.000000,9.333333,20.0,12.0,...,2,1,3,3,164.48,106.85,271.33,628.67,900.00,49.0
1,5,2237,354,1,1,4,11.000000,9.333333,20.0,12.0,...,2,1,3,3,164.48,106.85,271.33,628.67,900.00,48.0
2,5,2216,353,1,1,4,10.222222,9.333333,20.0,12.0,...,2,1,3,4,164.48,106.85,271.33,645.94,917.27,45.0
3,3,2968,351,3,1,3,2.300000,2.666667,2.0,2.5,...,2,1,3,4,4272.00,900.00,5172.00,9215.18,14387.18,11.0
4,5,2209,361,1,1,4,10.300000,9.333333,20.0,12.0,...,2,1,3,4,164.48,106.85,271.33,645.94,917.27,44.0


In [66]:
x = churn.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
churn_normalized = pd.DataFrame(x_scaled)
churn_normalized.columns = ['timezone_cd', 'tenure_date_created', 'tenure_visit', 'branch', 'abc_cd', 'abc_detailed_cd', 'avg_no_main_cat_1y', 'avg_no_main_cat_q1', 'avg_no_main_cat_q2', 'avg_no_main_cat_th', 'avg_no_main_cat_lh', 'avg_articles_6_6', 'avg_articles_q1_q2', 'churn_flag', 'growth_flag', 'decline_flag', 'ret_visits_per_chg_q1_q2', 'ret_sales_per_chg_q1_q2', 'visit_gap_ratio_th', 'visit_gap_ratio_1y', 'visit_gap_ratio_per_chg_6_6', 'avg_days_between_visits', 'home_st_visits_per_q1', 'home_st_visits_per_6_6', 'basket_spend', 'basket_spend_per_chg_6_6', 'basket_spend_per_chg_q1_q2', 'f_sales_per_chg_6_6', 'nf_visits_per_chg_6_6', 'nf_sales_per_chg_6_6', 'p_visits_per_chg_6_6', 'p_sales_per_chg_6_6', 'ret_visits_per_chg_6_6', 'ret_sales_per_chg_6_6', 'f_visits_per_chg_q1_q2', 'nf_visits_per_chg_q1_q2', 'nf_sales_per_chg_q1_q2', 'p_visits_per_chg_q1_q2', 'p_sales_per_chg_q1_q2', 'consistency_mnth_cd', 'consistency_qtr_cd', 'distinct_weeks_bought', 'expect_visit_flag', 'expect_visit_flag2', 'promo_sales_per', 'margin_per', 'coupon_disc_per', 'food_visits_per', 'food_sales_per', 'food_promo_per', 'nf_promo_per', 'food_colli_per', 'food_pieces_per', 'sales_last_model_period_per', 'visits_last_model_period_per', 'distinct_stores', 'recency', 'coupon_disc', 'margin_1y', 'home_visits_q1', 'home_visits_lh', 'home_visits_th', 'visit_gap_th', 'visit_gap_lh', 'visit_gap_1y', 'promo_sales_1y', 'promo_sales_q1', 'promo_sales_q2', 'promo_sales_th', 'promo_sales_lh', 'nf_promo_sales', 'food_promo_sales', 'promo_visits_q1', 'promo_visits_q2', 'promo_visits_th', 'promo_visits_lh', 'promo_visits_1y', 'nf_sales_1y', 'nf_sales_q1', 'nf_sales_q2', 'nf_sales_th', 'nf_sales_lh', 'nf_visits_q1', 'nf_visits_q2', 'nf_visits_th', 'nf_visits_lh', 'food_sales_1y', 'food_sales_th', 'food_sales_lh', 'food_visits_1y', 'food_visits_q1', 'food_visits_q2', 'pieces', 'food_colli', 'food_pieces', 'colli_1y', 'colli_q1', 'home_store_visits_q1', 'home_store_visits_th', 'home_store_visits_lh', 'visits_q1', 'visits_q2', 'visits_q3', 'visits_q4', 'visits_1y', 'visits_th', 'visits_lh', 'sales_1y', 'sales_q1', 'sales_q2', 'sales_q3', 'sales_q4', 'sales_th', 'sales_lh', 'ret_visits_q1', 'ret_visits_q2', 'ret_visits_th', 'ret_visits_lh', 'ret_sales_q1', 'ret_sales_q2', 'ret_sales_th', 'ret_sales_lh', 'ret_sales_1y', 'prediction_week']
churn_normalized.head()

,timezone_cd,tenure_date_created,tenure_visit,branch,abc_cd,abc_detailed_cd,avg_no_main_cat_1y,avg_no_main_cat_q1,avg_no_main_cat_q2,avg_no_main_cat_th,...,ret_visits_q1,ret_visits_q2,ret_visits_th,ret_visits_lh,ret_sales_q1,ret_sales_q2,ret_sales_th,ret_sales_lh,ret_sales_1y,prediction_week
0,0.75,0.145109,0.966667,0.0,0.0,0.428571,0.182884,0.124020,0.294004,0.177525,...,0.018519,0.0,0.017857,0.051724,0.003432,0.003607,0.005642,0.020444,0.018756,0.923077
1,0.75,0.144593,0.888889,0.0,0.0,0.428571,0.182884,0.124020,0.294004,0.177525,...,0.018519,0.0,0.017857,0.051724,0.003432,0.003607,0.005642,0.020444,0.018756,0.903846
2,0.75,0.143047,0.877778,0.0,0.0,0.428571,0.167982,0.124020,0.294004,0.177525,...,0.018519,0.0,0.017857,0.068966,0.003432,0.003607,0.005642,0.021006,0.019116,0.846154
3,0.50,0.198438,0.855556,1.0,0.0,0.285714,0.016198,0.020277,0.015474,0.019676,...,0.018519,0.0,0.017857,0.068966,0.089152,0.030382,0.107907,0.299673,0.300134,0.192308
4,0.75,0.142531,0.966667,0.0,0.0,0.428571,0.169472,0.124020,0.294004,0.177525,...,0.018519,0.0,0.017857,0.068966,0.003432,0.003607,0.005642,0.021006,0.019116,0.826923


In [67]:
all = ['timezone_cd', 'tenure_date_created', 'tenure_visit', 'branch', 'abc_cd', 'abc_detailed_cd', 'avg_no_main_cat_1y', 'avg_no_main_cat_q1', 'avg_no_main_cat_q2', 'avg_no_main_cat_th', 'avg_no_main_cat_lh', 'avg_articles_6_6', 'avg_articles_q1_q2', 'churn_flag', 'growth_flag', 'decline_flag', 'ret_visits_per_chg_q1_q2', 'ret_sales_per_chg_q1_q2', 'visit_gap_ratio_th', 'visit_gap_ratio_1y', 'visit_gap_ratio_per_chg_6_6', 'avg_days_between_visits', 'home_st_visits_per_q1', 'home_st_visits_per_6_6', 'basket_spend', 'basket_spend_per_chg_6_6', 'basket_spend_per_chg_q1_q2', 'f_sales_per_chg_6_6', 'nf_visits_per_chg_6_6', 'nf_sales_per_chg_6_6', 'p_visits_per_chg_6_6', 'p_sales_per_chg_6_6', 'ret_visits_per_chg_6_6', 'ret_sales_per_chg_6_6', 'f_visits_per_chg_q1_q2', 'nf_visits_per_chg_q1_q2', 'nf_sales_per_chg_q1_q2', 'p_visits_per_chg_q1_q2', 'p_sales_per_chg_q1_q2', 'consistency_mnth_cd', 'consistency_qtr_cd', 'distinct_weeks_bought', 'expect_visit_flag', 'expect_visit_flag2', 'promo_sales_per', 'margin_per', 'coupon_disc_per', 'food_visits_per', 'food_sales_per', 'food_promo_per', 'nf_promo_per', 'food_colli_per', 'food_pieces_per', 'sales_last_model_period_per', 'visits_last_model_period_per', 'distinct_stores', 'recency', 'coupon_disc', 'margin_1y', 'home_visits_q1', 'home_visits_lh', 'home_visits_th', 'visit_gap_th', 'visit_gap_lh', 'visit_gap_1y', 'promo_sales_1y', 'promo_sales_q1', 'promo_sales_q2', 'promo_sales_th', 'promo_sales_lh', 'nf_promo_sales', 'food_promo_sales', 'promo_visits_q1', 'promo_visits_q2', 'promo_visits_th', 'promo_visits_lh', 'promo_visits_1y', 'nf_sales_1y', 'nf_sales_q1', 'nf_sales_q2', 'nf_sales_th', 'nf_sales_lh', 'nf_visits_q1', 'nf_visits_q2', 'nf_visits_th', 'nf_visits_lh', 'food_sales_1y', 'food_sales_th', 'food_sales_lh', 'food_visits_1y', 'food_visits_q1', 'food_visits_q2', 'pieces', 'food_colli', 'food_pieces', 'colli_1y', 'colli_q1', 'home_store_visits_q1', 'home_store_visits_th', 'home_store_visits_lh', 'visits_q1', 'visits_q2', 'visits_q3', 'visits_q4', 'visits_1y', 'visits_th', 'visits_lh', 'sales_1y', 'sales_q1', 'sales_q2', 'sales_q3', 'sales_q4', 'sales_th', 'sales_lh', 'ret_visits_q1', 'ret_visits_q2', 'ret_visits_th', 'ret_visits_lh', 'ret_sales_q1', 'ret_sales_q2', 'ret_sales_th', 'ret_sales_lh', 'ret_sales_1y', 'prediction_week']
used_metro_model = ['margin_per', 'prediction_week', 'basket_spend', 'food_promo_per', 'promo_sales_per', 'recency', 'avg_no_main_cat_1y', 'nf_promo_per', 'visit_gap_th', 'avg_articles_6_6', 'nf_sales_1y', 'avg_no_main_cat_th', 'food_pieces_per', 'avg_no_main_cat_q1', 'visit_gap_ratio_per_chg_6_6', 'avg_articles_q1_q2', 'basket_spend_per_chg_q1_q2', 'food_sales_per', 'visit_gap_1y', 'basket_spend_per_chg_6_6', 'nf_sales_q1', 'nf_promo_sales', 'food_colli_per', 'tenure_visit', 'nf_sales_per_chg_6_6', 'ret_sales_1y', 'nf_sales_th', 'sales_q1', 'colli_q1', 'visit_gap_ratio_th', 'pieces', 'avg_no_main_cat_lh', 'sales_last_model_period_per', 'ret_sales_per_chg_6_6', 'p_sales_per_chg_6_6', 'ret_sales_th', 'nf_sales_lh', 'promo_sales_q1', 'ret_sales_q1', 'avg_no_main_cat_q2', 'p_sales_per_chg_q1_q2', 'nf_sales_per_chg_q1_q2', 'promo_sales_1y', 'f_sales_per_chg_6_6', 'margin_1y', 'ret_sales_lh', 'food_pieces', 'visit_gap_lh', 'f_visits_per_chg_q1_q2', 'visits_last_model_period_per', 'food_promo_sales', 'nf_sales_q2', 'nf_visits_per_chg_6_6', 'ret_sales_q2', 'visit_gap_ratio_1y', 'promo_sales_th', 'promo_sales_lh', 'promo_sales_q2', 'home_st_visits_per_6_6', 'p_visits_per_chg_6_6', 'p_visits_per_chg_q1_q2', 'avg_days_between_visits', 'colli_1y', 'ret_visits_per_chg_6_6', 'food_colli', 'sales_1y', 'sales_q2', 'nf_visits_per_chg_q1_q2', 'sales_q3', 'distinct_weeks_bought', 'ret_sales_per_chg_q1_q2', 'sales_q4', 'nf_visits_lh', 'abc_detailed_cd', 'food_sales_1y', 'food_sales_th', 'promo_visits_1y', 'home_visits_lh', 'home_visits_th', 'sales_lh', 'sales_th', 'visits_q3', 'ret_visits_lh', 'food_sales_lh', 'timezone_cd', 'visits_1y', 'food_visits_1y', 'nf_visits_th', 'promo_visits_lh', 'promo_visits_th', 'ret_visits_per_chg_q1_q2', 'home_visits_q1', 'visits_q4', 'visits_lh', 'nf_visits_q1', 'ret_visits_th', 'home_st_visits_per_q1', 'promo_visits_q1', 'visits_q1', 'branch', 'promo_visits_q2', 'nf_visits_q2', 'food_visits_q1', 'home_store_visits_lh', 'visits_th', 'visits_q2', 'food_visits_q2', 'home_store_visits_th', 'distinct_stores', 'ret_visits_q2', 'ret_visits_q1', 'home_store_visits_q1', 'consistency_mnth_cd', 'abc_cd', 'expect_visit_flag', 'expect_visit_flag2', 'consistency_qtr_cd']
for i in range(len(all)):
  if all[i] not in used_metro_model:
    print(all[i])
print(len(all))

tenure_date_created
churn_flag
growth_flag
decline_flag
coupon_disc_per
food_visits_per
coupon_disc
124


In [68]:
churn_dropped = churn_normalized
churn_dropped = churn_normalized.drop(['tenure_date_created', 'growth_flag', 'decline_flag', 'coupon_disc_per', 'food_visits_per', 'coupon_disc'], axis = 1)
churn_dropped.head()

,timezone_cd,tenure_visit,branch,abc_cd,abc_detailed_cd,avg_no_main_cat_1y,avg_no_main_cat_q1,avg_no_main_cat_q2,avg_no_main_cat_th,avg_no_main_cat_lh,...,ret_visits_q1,ret_visits_q2,ret_visits_th,ret_visits_lh,ret_sales_q1,ret_sales_q2,ret_sales_th,ret_sales_lh,ret_sales_1y,prediction_week
0,0.75,0.966667,0.0,0.0,0.428571,0.182884,0.124020,0.294004,0.177525,0.177827,...,0.018519,0.0,0.017857,0.051724,0.003432,0.003607,0.005642,0.020444,0.018756,0.923077
1,0.75,0.888889,0.0,0.0,0.428571,0.182884,0.124020,0.294004,0.177525,0.177827,...,0.018519,0.0,0.017857,0.051724,0.003432,0.003607,0.005642,0.020444,0.018756,0.903846
2,0.75,0.877778,0.0,0.0,0.428571,0.167982,0.124020,0.294004,0.177525,0.154116,...,0.018519,0.0,0.017857,0.068966,0.003432,0.003607,0.005642,0.021006,0.019116,0.846154
3,0.50,0.855556,1.0,0.0,0.285714,0.016198,0.020277,0.015474,0.019676,0.023052,...,0.018519,0.0,0.017857,0.068966,0.089152,0.030382,0.107907,0.299673,0.300134,0.192308
4,0.75,0.966667,0.0,0.0,0.428571,0.169472,0.124020,0.294004,0.177525,0.161361,...,0.018519,0.0,0.017857,0.068966,0.003432,0.003607,0.005642,0.021006,0.019116,0.826923


In [69]:
#move churn_flag
churn_reordered = churn_dropped[['churn_flag', 'timezone_cd', 'tenure_visit', 'branch', 'abc_cd', 'abc_detailed_cd', 'avg_no_main_cat_1y', 'avg_no_main_cat_q1', 'avg_no_main_cat_q2', 'avg_no_main_cat_th', 'avg_no_main_cat_lh', 'avg_articles_6_6', 'avg_articles_q1_q2', 'ret_visits_per_chg_q1_q2', 'ret_sales_per_chg_q1_q2', 'visit_gap_ratio_th', 'visit_gap_ratio_1y', 'visit_gap_ratio_per_chg_6_6', 'avg_days_between_visits', 'home_st_visits_per_q1', 'home_st_visits_per_6_6', 'basket_spend', 'basket_spend_per_chg_6_6', 'basket_spend_per_chg_q1_q2', 'f_sales_per_chg_6_6', 'nf_visits_per_chg_6_6', 'nf_sales_per_chg_6_6', 'p_visits_per_chg_6_6', 'p_sales_per_chg_6_6', 'ret_visits_per_chg_6_6', 'ret_sales_per_chg_6_6', 'f_visits_per_chg_q1_q2', 'nf_visits_per_chg_q1_q2', 'nf_sales_per_chg_q1_q2', 'p_visits_per_chg_q1_q2', 'p_sales_per_chg_q1_q2', 'consistency_mnth_cd', 'consistency_qtr_cd', 'distinct_weeks_bought', 'expect_visit_flag', 'expect_visit_flag2', 'promo_sales_per', 'margin_per', 'food_sales_per', 'food_promo_per', 'nf_promo_per', 'food_colli_per', 'food_pieces_per', 'sales_last_model_period_per', 'visits_last_model_period_per', 'distinct_stores', 'recency', 'margin_1y', 'home_visits_q1', 'home_visits_lh', 'home_visits_th', 'visit_gap_th', 'visit_gap_lh', 'visit_gap_1y', 'promo_sales_1y', 'promo_sales_q1', 'promo_sales_q2', 'promo_sales_th', 'promo_sales_lh', 'nf_promo_sales', 'food_promo_sales', 'promo_visits_q1', 'promo_visits_q2', 'promo_visits_th', 'promo_visits_lh', 'promo_visits_1y', 'nf_sales_1y', 'nf_sales_q1', 'nf_sales_q2', 'nf_sales_th', 'nf_sales_lh', 'nf_visits_q1', 'nf_visits_q2', 'nf_visits_th', 'nf_visits_lh', 'food_sales_1y', 'food_sales_th', 'food_sales_lh', 'food_visits_1y', 'food_visits_q1', 'food_visits_q2', 'pieces', 'food_colli', 'food_pieces', 'colli_1y', 'colli_q1', 'home_store_visits_q1', 'home_store_visits_th', 'home_store_visits_lh', 'visits_q1', 'visits_q2', 'visits_q3', 'visits_q4', 'visits_1y', 'visits_th', 'visits_lh', 'sales_1y', 'sales_q1', 'sales_q2', 'sales_q3', 'sales_q4', 'sales_th', 'sales_lh', 'ret_visits_q1', 'ret_visits_q2', 'ret_visits_th', 'ret_visits_lh', 'ret_sales_q1', 'ret_sales_q2', 'ret_sales_th', 'ret_sales_lh', 'ret_sales_1y', 'prediction_week']]
churn_reordered.head()

,churn_flag,timezone_cd,tenure_visit,branch,abc_cd,abc_detailed_cd,avg_no_main_cat_1y,avg_no_main_cat_q1,avg_no_main_cat_q2,avg_no_main_cat_th,...,ret_visits_q1,ret_visits_q2,ret_visits_th,ret_visits_lh,ret_sales_q1,ret_sales_q2,ret_sales_th,ret_sales_lh,ret_sales_1y,prediction_week
0,1.0,0.75,0.966667,0.0,0.0,0.428571,0.182884,0.124020,0.294004,0.177525,...,0.018519,0.0,0.017857,0.051724,0.003432,0.003607,0.005642,0.020444,0.018756,0.923077
1,1.0,0.75,0.888889,0.0,0.0,0.428571,0.182884,0.124020,0.294004,0.177525,...,0.018519,0.0,0.017857,0.051724,0.003432,0.003607,0.005642,0.020444,0.018756,0.903846
2,1.0,0.75,0.877778,0.0,0.0,0.428571,0.167982,0.124020,0.294004,0.177525,...,0.018519,0.0,0.017857,0.068966,0.003432,0.003607,0.005642,0.021006,0.019116,0.846154
3,0.0,0.50,0.855556,1.0,0.0,0.285714,0.016198,0.020277,0.015474,0.019676,...,0.018519,0.0,0.017857,0.068966,0.089152,0.030382,0.107907,0.299673,0.300134,0.192308
4,1.0,0.75,0.966667,0.0,0.0,0.428571,0.169472,0.124020,0.294004,0.177525,...,0.018519,0.0,0.017857,0.068966,0.003432,0.003607,0.005642,0.021006,0.019116,0.826923


In [70]:
# Separate majority and minority classes
churn_majority = churn_reordered[churn_reordered.churn_flag==0]
churn_minority = churn_reordered[churn_reordered.churn_flag==1]

In [71]:
# Upsample minority class
churn_minority_upsampled = resample(churn_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=10000,    # to match majority class
                                 random_state=158) # reproducible results
 
# Combine majority class with upsampled minority class
churn_upsampled = pd.concat([churn_majority, churn_minority_upsampled])
 
# Display new class counts
churn_upsampled.churn_flag.value_counts()
# 1    576
# 0    576
# Name: balance, dtype: int64

1.0    10000
0.0     9806
Name: churn_flag, dtype: int64

In [72]:
churn_upsampled.head()

,churn_flag,timezone_cd,tenure_visit,branch,abc_cd,abc_detailed_cd,avg_no_main_cat_1y,avg_no_main_cat_q1,avg_no_main_cat_q2,avg_no_main_cat_th,...,ret_visits_q1,ret_visits_q2,ret_visits_th,ret_visits_lh,ret_sales_q1,ret_sales_q2,ret_sales_th,ret_sales_lh,ret_sales_1y,prediction_week
3,0.0,0.5,0.855556,1.0,0.0,0.285714,0.016198,0.020277,0.015474,0.019676,...,0.018519,0.0,0.017857,0.068966,0.089152,0.030382,0.107907,0.299673,0.300134,0.192308
6,0.0,0.0,0.766667,0.0,1.0,0.714286,0.170567,0.139581,0.448743,0.231526,...,0.000000,0.0,0.000000,0.017241,0.000167,0.001688,0.001190,0.000806,0.001707,0.673077
7,0.0,0.0,0.700000,0.0,0.0,0.428571,0.193102,0.129207,0.185687,0.152602,...,0.000000,0.0,0.000000,0.000000,0.004896,0.004734,0.007802,0.000000,0.007800,0.230769
8,0.0,0.0,0.777778,0.0,0.0,0.428571,0.193102,0.129207,0.185687,0.152602,...,0.000000,0.0,0.000000,0.000000,0.004896,0.004734,0.007802,0.000000,0.007800,0.250000
11,0.0,0.0,0.688889,0.0,1.0,0.714286,0.170567,0.139581,0.448743,0.231526,...,0.000000,0.0,0.000000,0.017241,0.000167,0.001688,0.001190,0.000806,0.001707,0.653846


In [73]:
shuffled = churn_upsampled.sample(frac=1)
trainsize = int(len(shuffled['churn_flag']) * 0.75)
validsize = int(len(shuffled['churn_flag']) * 0.25)

churn_train = shuffled.iloc[:trainsize, :]
churn_valid = shuffled.iloc[trainsize:(trainsize+validsize), :]
churn_test = churn_valid
print(churn_train.churn_flag.value_counts())
print(churn_valid.churn_flag.value_counts())

1.0    7519
0.0    7335
Name: churn_flag, dtype: int64
1.0    2481
0.0    2470
Name: churn_flag, dtype: int64


In [76]:
def to_csv(df, filename):
  outdf = df.copy(deep=False)
  
  # reorder columns so that target is first column
  cols = outdf.columns.tolist()
  print (cols)  # new order of columns
  outdf = outdf[cols]
  outdf.to_csv(filename, header=False, index_label=False, index=False)
  
  print outdf.shape

to_csv(churn_train, 'churn-einzelhandel-train.csv')
to_csv(churn_valid, 'churn-einzelhandel-valid.csv')
to_csv(churn_test, 'churn-einzelhandel-test.csv')

['churn_flag', 'timezone_cd', 'tenure_visit', 'branch', 'abc_cd', 'abc_detailed_cd', 'avg_no_main_cat_1y', 'avg_no_main_cat_q1', 'avg_no_main_cat_q2', 'avg_no_main_cat_th', 'avg_no_main_cat_lh', 'avg_articles_6_6', 'avg_articles_q1_q2', 'ret_visits_per_chg_q1_q2', 'ret_sales_per_chg_q1_q2', 'visit_gap_ratio_th', 'visit_gap_ratio_1y', 'visit_gap_ratio_per_chg_6_6', 'avg_days_between_visits', 'home_st_visits_per_q1', 'home_st_visits_per_6_6', 'basket_spend', 'basket_spend_per_chg_6_6', 'basket_spend_per_chg_q1_q2', 'f_sales_per_chg_6_6', 'nf_visits_per_chg_6_6', 'nf_sales_per_chg_6_6', 'p_visits_per_chg_6_6', 'p_sales_per_chg_6_6', 'ret_visits_per_chg_6_6', 'ret_sales_per_chg_6_6', 'f_visits_per_chg_q1_q2', 'nf_visits_per_chg_q1_q2', 'nf_sales_per_chg_q1_q2', 'p_visits_per_chg_q1_q2', 'p_sales_per_chg_q1_q2', 'consistency_mnth_cd', 'consistency_qtr_cd', 'distinct_weeks_bought', 'expect_visit_flag', 'expect_visit_flag2', 'promo_sales_per', 'margin_per', 'food_sales_per', 'food_promo_per',

In [75]:
data = "churn-einzelhandel-test.csv"
names = ['timezone_cd', 'tenure_visit', 'branch', 'abc_cd', 'abc_detailed_cd', 'avg_no_main_cat_1y', 'avg_no_main_cat_q1', 'avg_no_main_cat_q2', 'avg_no_main_cat_th', 'avg_no_main_cat_lh', 'avg_articles_6_6', 'avg_articles_q1_q2', 'churn_flag', 'ret_visits_per_chg_q1_q2', 'ret_sales_per_chg_q1_q2', 'visit_gap_ratio_th', 'visit_gap_ratio_1y', 'visit_gap_ratio_per_chg_6_6', 'avg_days_between_visits', 'home_st_visits_per_q1', 'home_st_visits_per_6_6', 'basket_spend', 'basket_spend_per_chg_6_6', 'basket_spend_per_chg_q1_q2', 'f_sales_per_chg_6_6', 'nf_visits_per_chg_6_6', 'nf_sales_per_chg_6_6', 'p_visits_per_chg_6_6', 'p_sales_per_chg_6_6', 'ret_visits_per_chg_6_6', 'ret_sales_per_chg_6_6', 'f_visits_per_chg_q1_q2', 'nf_visits_per_chg_q1_q2', 'nf_sales_per_chg_q1_q2', 'p_visits_per_chg_q1_q2', 'p_sales_per_chg_q1_q2', 'consistency_mnth_cd', 'consistency_qtr_cd', 'distinct_weeks_bought', 'expect_visit_flag', 'expect_visit_flag2', 'promo_sales_per', 'margin_per', 'food_sales_per', 'food_promo_per', 'nf_promo_per', 'food_colli_per', 'food_pieces_per', 'sales_last_model_period_per', 'visits_last_model_period_per', 'distinct_stores', 'recency', 'margin_1y', 'home_visits_q1', 'home_visits_lh', 'home_visits_th', 'visit_gap_th', 'visit_gap_lh', 'visit_gap_1y', 'promo_sales_1y', 'promo_sales_q1', 'promo_sales_q2', 'promo_sales_th', 'promo_sales_lh', 'nf_promo_sales', 'food_promo_sales', 'promo_visits_q1', 'promo_visits_q2', 'promo_visits_th', 'promo_visits_lh', 'promo_visits_1y', 'nf_sales_1y', 'nf_sales_q1', 'nf_sales_q2', 'nf_sales_th', 'nf_sales_lh', 'nf_visits_q1', 'nf_visits_q2', 'nf_visits_th', 'nf_visits_lh', 'food_sales_1y', 'food_sales_th', 'food_sales_lh', 'food_visits_1y', 'food_visits_q1', 'food_visits_q2', 'pieces', 'food_colli', 'food_pieces', 'colli_1y', 'colli_q1', 'home_store_visits_q1', 'home_store_visits_th', 'home_store_visits_lh', 'visits_q1', 'visits_q2', 'visits_q3', 'visits_q4', 'visits_1y', 'visits_th', 'visits_lh', 'sales_1y', 'sales_q1', 'sales_q2', 'sales_q3', 'sales_q4', 'sales_th', 'sales_lh', 'ret_visits_q1', 'ret_visits_q2', 'ret_visits_th', 'ret_visits_lh', 'ret_sales_q1', 'ret_sales_q2', 'ret_sales_th', 'ret_sales_lh', 'ret_sales_1y', 'prediction_week']  
df = pd.read_csv(data, names=names, skiprows=0)  
df.describe()

,timezone_cd,tenure_visit,branch,abc_cd,abc_detailed_cd,avg_no_main_cat_1y,avg_no_main_cat_q1,avg_no_main_cat_q2,avg_no_main_cat_th,avg_no_main_cat_lh,...,ret_visits_q1,ret_visits_q2,ret_visits_th,ret_visits_lh,ret_sales_q1,ret_sales_q2,ret_sales_th,ret_sales_lh,ret_sales_1y,prediction_week
count,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,...,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000
mean,0.278200,0.881147,0.338800,0.911200,0.841943,0.303313,0.240133,0.261906,0.264265,0.323387,...,0.030096,0.135378,0.050779,0.084697,0.004387,0.005459,0.007742,0.010691,0.014599,0.483477
std,0.284473,0.158471,0.449995,0.284512,0.199952,0.148304,0.127164,0.133713,0.132593,0.160201,...,0.036963,0.159982,0.051354,0.078038,0.019508,0.029449,0.029690,0.046660,0.051028,0.298775
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000871,0.000000,0.000000,0.000875,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000007,0.000000,0.000036,0.000000
25%,0.000000,0.855556,0.000000,1.000000,0.714286,0.203508,0.151964,0.167118,0.170879,0.214636,...,0.000000,0.000000,0.017857,0.034483,0.000194,0.000270,0.000788,0.000710,0.002176,0.225962
50%,0.250000,0.933333,0.000000,1.000000,0.857143,0.284846,0.223379,0.244144,0.248577,0.299742,...,0.018519,0.111111,0.035714,0.068966,0.000831,0.001200,0.002419,0.002593,0.004958,0.461538
75%,0.500000,0.966667,1.000000,1.000000,1.000000,0.388526,0.310458,0.338284,0.335046,0.416331,...,0.037037,0.222222,0.071429,0.103448,0.002845,0.004218,0.005676,0.007396,0.010141,0.716346
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.949710,0.970923,0.975082,...,0.388889,1.000000,0.464286,1.000000,0.289120,1.000000,0.662590,0.978592,0.759595,1.000000


In [ ]:
#Downsample majority classPython

# Separate majority and minority classes
df_majority = df[df.balance==0]
df_minority = df[df.balance==1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=49,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.balance.value_counts()
# 1    49
# 0    49
# Name: balance, dtype: int64

In [60]:
from sklearn.metrics import *
import random

y_pred = [random.randint(0, 1) for i in range(100)]
print(type(y_pred))
y_true = [random.randint(0, 1) for i in range(100)]

print precision_score(y_true, y_pred)
print recall_score(y_true, y_pred)
print f1_score(y_true, y_pred)
print accuracy_score(y_true, y_pred)

<type 'list'>
0.3829787234042553
0.375
0.37894736842105264
0.41


In [77]:
print(type(tf.Session().run(tf.constant([1,2,3]))))

<type 'numpy.ndarray'>
